In [1]:
#import types
from git import Repo
import os
import numpy as np
import xarray as xr
import pandas as pd
import scipy
import Nio
import datetime
import sys

script_full_path = '/scripts/convert_grb_to_netcdf.py'

# Process TAR models

In [2]:
# Second Assessment Report (SAR) Model Output

dir_path = "../data/raw/SAR/"

def add_metadata(ds,run_name):
    metadata_path = '../data/metadata/sar_models.csv'
    data = pd.read_csv(metadata_path,sep=',')
    ds.attrs['run_description'] = np.str(
        data['run_description'][list(data['run_name']).index(run_name.replace('CI01','GS01'))]
    ).replace('nan','N/A')
    
    ds.attrs['model_description'] = np.str(
        data['model_description'][list(data['run_name']).index(run_name.replace('CI01','GS01'))]
    ).replace('nan','N/A')
    return ds

def date_to_datetime(dates,reference_date):
    date = np.array(
        [datestr.split(" ")[0].split("/")[2] + "-" +
         datestr.split(" ")[0].split("/")[0] + "-" +
         datestr.split(" ")[0].split("/")[1]
         for datestr in np.array(dates.values).astype("str")]
    ).astype("datetime64")
    return (date-reference_date)/np.array([1]).astype("<m8[D]")

# Main loop
nexp = 0
for institution in os.listdir(dir_path):
    if ("." in institution): continue
    for file_name in os.listdir(dir_path+institution+"/"):
        nexp+=1
print("Total # of experiments: "+str(nexp))

# Variable metadata copied from http://cfconventions.org/Data/cf-standard-names/27/build/cf-standard-name-table.html
standard_dict = {}
standard_dict['long_name'] = {
    'tas': 'Near-Surface Air Temperature',
    'psl': 'Sea Level Pressure',
    'pr': 'Precipitation',
    'rsds': 'Downwelling Shortwave Flux at Surface',
    'sn': 'Snow Amount',
    'tasmax': 'Maximum Near-Surface Air Temperature',
    'tasmin': 'Minimum Near-Surface Air Temperature',
    'sfcWind': 'Near-Surface Wind Speed'
}
standard_dict['description'] = {
    'tas': 'temperature at 2-meter height',
    'psl': 'not, in general, the same as surface pressure',
    'pr': 'at surface; includes both liquid and solid phases from all types of clouds',
    'rsds': """
    The surface called "surface" means the lower boundary of the atmosphere. 
    "shortwave" means shortwave radiation. Downwelling radiation is radiation from above.
    It does not mean "net downward". Surface downwelling shortwave is the sum of direct
    and diffuse solar radiation incident on the surface, and is sometimes called "global
    radiation". When thought of as being incident on a surface, a radiative flux is
    sometimes called "irradiance". In addition, it is identical with the quantity measured
    by a cosine-collector light-meter and sometimes called "vector irradiance". In
    accordance with common usage in geophysical disciplines, "flux" implies
    per unit area, called "flux density" in physics.
    """,
    'sn': '"Amount" means mass per unit area.',
    'tasmax': 'Monthly-mean daily-maximum temperature at 2-meter height',
    'tasmin': 'Monthly-mean daily-minimum temperature at 2-meter height',
    'sfcWind': """
    'Speed is the magnitude of velocity. Wind is defined as a two-dimensional (horizontal)
    air velocity vector, with no vertical component. (Vertical motion in the atmosphere has
    the standard name upward_air_velocity.) The wind speed is the magnitude of the wind
    velocity.'
    """
}
standard_dict['standard_name'] = {
    'tas': 'air_temperature',
    'psl': 'air_pressure_at_sea_level',
    'pr': 'precipitation_flux',
    'rsds': 'surface_downwelling_shortwave_flux',
    'sn': 'snow_amount',
    'tasmax': 'air_temperature',
    'tasmin': 'air_temperature',
    'sfcWind': 'wind_speed'
}
standard_dict['units'] = {
    'tas': 'K',
    'psl': 'Pa',
    'pr': 'kg m^-2 s^-1',
    'rsds': 'W m^-2',
    'sn': 'kg m^-2',
    'tasmax': 'K',
    'tasmin': 'K',
    'sfcWind': 'm s^-1'
}

for institution in os.listdir(dir_path):
    if ("." == institution[0]): continue
    stop = 0
    for var_name in os.listdir(dir_path+institution+"/"):
        if ("." == var_name[0]): continue
        for file_name in os.listdir(dir_path+institution+"/"+var_name+"/"):
            if ("." == file_name[0]): continue
                
            run_name = str.split(file_name,"_")[0]
            
            print("\n"+institution+"/"+var_name+"/"+file_name, end="")

            # Load data into xarray dataset using PyNio engine
            ds = xr.open_dataset(dir_path+institution+"/"+var_name+"/"+file_name,engine="pynio",decode_times=False)
            
            # Make coordinates CF-compliant and convert dates to decimal years
            reference_date = np.datetime64("1990","D")
            date = date_to_datetime(ds['initial_time0'],reference_date)
            var_change_dict = {}
            var_change_dict[list(ds.dims)[0]] = 'latitude'
            var_change_dict[list(ds.dims)[1]] = 'longitude'
            var_change_dict[list(ds.dims)[2]] = 'time'
            ds = ds.rename(var_change_dict)
            ds.coords['latitude'].attrs['axis']='Y'
            ds.coords['latitude'].attrs['standard_name'] = 'latitude'
            ds.coords['longitude'].attrs['axis']='X'
            ds.coords['longitude'].attrs['standard_name'] = 'longitude'
            ds.coords['time'].attrs['long_name'] = 'time'
            ds.coords['time'].attrs['axis'] = 'T'
            ds.coords['time'].attrs['calendar'] = 'standard'
            ds.coords['time'].values = date
            ds.coords['time'].attrs['units'] = 'days since 1990-01-01 0:0:0'
            ds = ds.drop(['initial_time0_encoded','initial_time0'])

            # Give temperature variable to standard names and description
            var_names = ds.variables.keys()
            for nam in var_names:
                if not(("latitude" in nam) or ("longitude" in nam) or ("time" in nam)):
                    ds = ds.rename({nam:var_name})

            # Convert precipitation data from kg/m^2/day (is this really the units?) to kg/m^2/s
            if var_name == "pr":
                ds[var_name] /= (24.*60.*60.)
                        
            for attrs_name in list(standard_dict.keys()):
                ds[var_name].attrs[attrs_name] = standard_dict[attrs_name][var_name]
                
            ds[var_name] = ds[var_name].where(ds[var_name]!=-999., np.nan)
            
            # Fix MPIfM
            if institution == "MPIfM":
                ds['latitude'].values = ds['latitude'].values[::-1]
            
            # Declare CF-convention compliance
            ds.attrs['Conventions'] = 'CF-1.7'

            # Metadata 
            ds.attrs['title'] = 'Projections from a Second Assessment Report model'
            ds.attrs['institution'] = institution
            ds.attrs['modelling_center'] = ds[var_name].attrs['center']
            if 'model' in ds.attrs:
                ds.attrs['source'] = ds[var_name].attrs['model']
            else: 
                ds.attrs['source'] = 'N/A'

            # Generate data provenance entry:
            # time stamp, command line arguments, environment, and hash for git commit
            time_stamp = datetime.datetime.now().strftime("%a %b %d %H:%M:%S %Y")
            args = " ".join(sys.argv)
            exe = sys.executable
            git_hash = Repo(os.getcwd()+'/..').head.commit.hexsha[0:7]
            history_entry = ("{time_stamp}: {exe} {args} {script_full_path} (Git hash: {git_hash})"
                     .format(time_stamp=time_stamp,
                             exe=exe,
                             args=args,
                             script_full_path=script_full_path,
                             git_hash = git_hash))
            ds.attrs['history'] = history_entry

            # Write xarray dataset to netCDF4 file
            ncfile_name = "../data/interim/SAR/"+run_name+"_"+var_name+".nc"

            ds.to_netcdf(ncfile_name,mode='w')
            ds.close()

Total # of experiments: 58

CCSR-NIES/sfcWind/NI01GG01_WNDS_1-2520.grb
CCSR-NIES/sfcWind/NI01GS01_WNDS_1-2520.grb
CCSR-NIES/sfcWind/NI01CI01_WNDS_1-2520.grb
CCSR-NIES/tas/NI01GS01_TA___1-2520.grb
CCSR-NIES/tas/NI01GG01_TA___1-2520.grb
CCSR-NIES/tas/NI01CI01_TA___1-2520.grb
CCSR-NIES/tasmin/NI01GG01_TMIN_1-2520.grb
CCSR-NIES/tasmin/NI01CI01_TMIN_1-2520.grb
CCSR-NIES/tasmin/NI01GS01_TMIN_1-2520.grb
CCSR-NIES/tasmax/NI01GS01_TMAX_1-2520.grb
CCSR-NIES/tasmax/NI01CI01_TMAX_1-2520.grb
CCSR-NIES/tasmax/NI01GG01_TMAX_1-2520.grb
CCSR-NIES/pr/NI01GS01_RAIN_1-2520.grb
CCSR-NIES/pr/NI01GG01_RAIN_1-2520.grb
CCSR-NIES/pr/NI01CI01_RAIN_1-2520.grb
CCSR-NIES/rsds/NI01GG01_SSR__1-2520.grb
CCSR-NIES/rsds/NI01CI01_SSR__1-2520.grb
CCSR-NIES/rsds/NI01GS01_SSR__1-2520.grb
CCSR-NIES/psl/NI01GS01_SLP__1-2520.grb
CCSR-NIES/psl/NI01CI01_SLP__1-2520.grb
CCSR-NIES/psl/NI01GG01_SLP__1-2520.grb
CSIRO/tas/CS01GG01_TAS__1-2640.grb
CSIRO/tas/CS01GS01_TAS__1-2640.grb
CSIRO/tas/CS01CI01_TAS__1-2640.grb
CSIRO/tasmin/CS01G

# Process TAR models

In [3]:
# Third Assessment Report (SAR) Model Output
dir_path = "../data/raw/TAR/"

# Main loop
nexp = 0
for institution in os.listdir(dir_path):
    if ("." in institution): continue
    for file_name in os.listdir(dir_path+institution+"/"):
        nexp+=1
print("\nTotal # of experiments: "+str(nexp))

for institution in os.listdir(dir_path):
    if ("." == institution[0]): continue
    stop = 0
    for var_name in os.listdir(dir_path+institution+"/"):
        if ("." == var_name[0]): continue
        for file_name in os.listdir(dir_path+institution+"/"+var_name+"/"):
            if ("." == file_name[0]): continue
                
            run_name = str.split(file_name,"_")[0]+"_"+str.split(file_name,"_")[1]+"-"+str.split(file_name,"_")[2]
            
            print("\n"+institution+"/"+var_name+"/"+file_name, end="")

            # Load data into xarray dataset using PyNio engine
            ds = xr.open_dataset(dir_path+institution+"/"+var_name+"/"+file_name,engine="pynio",decode_times=False)

            # Make coordinates CF-compliant and convert dates to decimal years
            reference_date = np.datetime64("1990","D")
            date = date_to_datetime(ds['initial_time0'],reference_date)
            var_change_dict = {}
            var_change_dict[list(ds.dims)[0]] = 'latitude'
            var_change_dict[list(ds.dims)[1]] = 'longitude'
            var_change_dict[list(ds.dims)[2]] = 'time'
            ds = ds.rename(var_change_dict)
            ds.coords['latitude'].attrs['axis']='Y'
            ds.coords['latitude'].attrs['standard_name'] = 'latitude'
            ds.coords['longitude'].attrs['axis']='X'
            ds.coords['longitude'].attrs['standard_name'] = 'longitude'
            ds.coords['time'].attrs['long_name'] = 'time'
            ds.coords['time'].attrs['axis'] = 'T'
            ds.coords['time'].attrs['calendar'] = 'standard'
            ds.coords['time'].values = date
            ds.coords['time'].attrs['units'] = 'days since 1990-01-01 0:0:0'
            ds = ds.drop(['initial_time0_encoded','initial_time0'])

            # Give temperature variable to standard names and description
            var_names = ds.variables.keys()
            for nam in var_names:
                if not(("latitude" in nam) or ("longitude" in nam) or ("time" in nam)):
                    ds = ds.rename({nam:var_name})

            for attrs_name in list(standard_dict.keys()):
                ds[var_name].attrs[attrs_name] = standard_dict[attrs_name][var_name]
                
            # Convert precipitation data from kg/m^2/day (is this really the units?) to kg/m^2/s
            if var_name == "pr":
                ds[var_name] /= (24.*60.*60.)
                
            ds[var_name] = ds[var_name].where(ds[var_name]!=-999., np.nan)

            # Fix EH4OPYC
            if institution == "MPIfM":
                ds['latitude'].values = ds['latitude'].values[::-1]
            
            # Declare CF-convention compliance
            ds.attrs['Conventions'] = 'CF-1.7'

            # Metadata 
            ds.attrs['title'] = 'Projections from a Third Assessment Report model'
            ds.attrs['institution'] = institution
            ds.attrs['modelling_center'] = ds[var_name].attrs['center']
            if 'model' in ds.attrs:
                ds.attrs['source'] = ds[var_name].attrs['model']
            else: 
                ds.attrs['source'] = 'N/A'

            #ds = add_metadata(ds,run_name)

            # Generate data provenance entry:
            # time stamp, command line arguments, environment, and hash for git commit
            time_stamp = datetime.datetime.now().strftime("%a %b %d %H:%M:%S %Y")
            args = " ".join(sys.argv)
            exe = sys.executable
            git_hash = Repo(os.getcwd()+'/..').head.commit.hexsha[0:7]
            history_entry = ("{time_stamp}: {exe} {args} {script_full_path} (Git hash: {git_hash})"
                     .format(time_stamp=time_stamp,
                             exe=exe,
                             args=args,
                             script_full_path=script_full_path,
                             git_hash = git_hash))
            ds.attrs['history'] = history_entry

            # Write xarray dataset to netCDF4 file
            ncfile_name = "../data/interim/TAR/"+run_name+"_"+var_name+".nc"
            ds.to_netcdf(ncfile_name,mode='w')
            ds.close()


Total # of experiments: 56

CCSR-NIES/sfcWind/CCSRNIES_SRES_B2_WIND10_1-2532.grb
CCSR-NIES/sfcWind/CCSRNIES_SRES_A2_WIND10_1-2532.grb
CCSR-NIES/tas/CCSRNIES_SRES_A2_TS_1-2532.grb
CCSR-NIES/tas/CCSRNIES_SRES_B2_TS_1-2532.grb
CCSR-NIES/tasmin/CCSRNIES_SRES_A2_TMIN_1-2532.grb
CCSR-NIES/tasmin/CCSRNIES_SRES_B2_TMIN_1-2532.grb
CCSR-NIES/tasmax/CCSRNIES_SRES_B2_TMAX_1-2532.grb
CCSR-NIES/tasmax/CCSRNIES_SRES_A2_TMAX_1-2532.grb
CCSR-NIES/pr/CCSRNIES_SRES_A2_PREC_1-2532.grb
CCSR-NIES/pr/CCSRNIES_SRES_B2_PREC_1-2532.grb
CCSR-NIES/rsds/CCSRNIES_SRES_A2_DSWF_1-2532.grb
CCSR-NIES/rsds/CCSRNIES_SRES_A2_DSWF_1-2532(1).grb
CCSR-NIES/rsds/CCSRNIES_SRES_B2_DSWF_1-2532.grb
CCSR-NIES/psl/CCSRNIES_SRES_A2_MSLP_1-2532.grb
CCSR-NIES/psl/CCSRNIES_SRES_B2_MSLP_1-2532.grb
CSIRO/sfcWind/CSIRO_SRES_A2_WIND10_1-1680.grb
CSIRO/sfcWind/CSIRO_SRES_B2_WIND10_1-1680.grb
CSIRO/tas/CSIRO_SRES_B2_TS_1-1680.grb
CSIRO/tas/CSIRO_SRES_A2_TS_1-1680.grb
CSIRO/tasmin/CSIRO_SRES_B2_TMIN_1-1680.grb
CSIRO/tasmin/CSIRO_SRES_A2_TMIN